In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import torch.nn.functional as F
from torch import nn
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import math

# manage ray's relative imports
import ray
runtime_env = {"working_dir": ".." }
ray.init(runtime_env=runtime_env, dashboard_port=13065, include_dashboard=True)

from ray import tune
from ray.tune.suggest.optuna import OptunaSearch
from ray.tune import JupyterNotebookReporter

# manage beams's relative imports
import sys
sys.path.append('..')

from src.beam import beam_arguments, Experiment, Study
from src.beam import UniversalDataset, UniversalBatchSampler, PackedFolds
from src.beam import Algorithm
from src.beam import LinearNet, check_type, slice_to_index
from torchvision import transforms
import matplotlib.pyplot as plt

from src.beam import DataTensor
from src.beam.utils import is_notebook

from sklearn.datasets import fetch_covtype
import pandas as pd

from covtype_example import CovtypeDataset, CovtypeAlgorithm, RuleNet, covtype_algorithm_generator, get_covtype_parser 

2022-06-26 12:28:11,979	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:13065
2022-06-26 12:28:12,512	INFO packaging.py:388 -- Creating a file package for local directory '..'.
2022-06-26 12:28:12,536	INFO packaging.py:241 -- Pushing file package 'gcs://_ray_pkg_16b7a439f752140a.zip' (1.03MiB) to Ray cluster...
2022-06-26 12:28:12,551	INFO packaging.py:243 -- Successfully pushed file package 'gcs://_ray_pkg_16b7a439f752140a.zip'.


In [2]:
path_to_data = '/home/shared/data/dataset/covtype'
root_dir = '/home/shared/data/results/covtype'

In [13]:
args = beam_arguments(get_covtype_parser(), 
        f"--project-name=covtype --root-dir={root_dir} --algorithm=CovtypeAlgorithm --device=0 --no-half --lr-d=1e-3 --lr-s=.02 --batch-size=512",
        "--n-epochs=100 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark",
        "--weight-decay=1e-5 --beta1=0.9 --beta2=0.99", weight_factor=1., scheduler_patience=16, weight_decay=1e-3,  label_smoothing=.2,
                      k_p=.05, k_i=0.001, k_d=0.005, initial_mask=1,
        path_to_data=path_to_data, dropout=.0, activation='gelu', channels=256, n_rules=128, n_layers=4, scheduler_factor=1/math.sqrt(10))

experiment = Experiment(args)

2022-06-26 12:44:48.913 | INFO     | src.beam.experiment:__init__:183 - beam project: covtype
2022-06-26 12:44:48.915 | INFO     | src.beam.experiment:__init__:184 - Experiment Hyperparameters
2022-06-26 12:44:48.916 | INFO     | src.beam.experiment:__init__:189 - project_name: covtype
2022-06-26 12:44:48.917 | INFO     | src.beam.experiment:__init__:189 - identifier: debug
2022-06-26 12:44:48.918 | INFO     | src.beam.experiment:__init__:189 - algorithm: CovtypeAlgorithm
2022-06-26 12:44:48.919 | INFO     | src.beam.experiment:__init__:189 - mp_port: None
2022-06-26 12:44:48.920 | INFO     | src.beam.experiment:__init__:189 - root_dir: /home/shared/data/results/covtype
2022-06-26 12:44:48.921 | INFO     | src.beam.experiment:__init__:189 - reload: False
2022-06-26 12:44:48.922 | INFO     | src.beam.experiment:__init__:189 - resume: -1
2022-06-26 12:44:48.923 | INFO     | src.beam.experiment:__init__:189 - override: False
2022-06-26 12:44:48.924 | INFO     | src.beam.experiment:__init_

In [37]:
args = beam_arguments(get_covtype_parser(), 
        f"--project-name=covtype --root-dir={root_dir} --algorithm=CovtypeAlgorithm --device=0 --clip=0. --amp --lr-d=1e-3 --lr-s=.02 --batch-size=1024",
        "--n-epochs=100 --parallel=1 --accumulate=1 --cudnn-benchmark",
        "--weight-decay=1e-5 --beta1=0.9 --beta2=0.999", weight_factor=.0, scheduler_patience=16,  label_smoothing=.1,
                      k_p=.05, k_i=0.001, k_d=0.005, initial_mask=1,
        path_to_data=path_to_data, dropout=.0, activation='gelu', channels=192, n_rules=128, n_layers=3, scheduler_factor=1/math.sqrt(10))

experiment = Experiment(args)

2022-06-26 15:27:13.751 | INFO     | src.beam.experiment:__init__:183 - beam project: covtype
2022-06-26 15:27:13.752 | INFO     | src.beam.experiment:__init__:184 - Experiment Hyperparameters
2022-06-26 15:27:13.753 | INFO     | src.beam.experiment:__init__:189 - project_name: covtype
2022-06-26 15:27:13.754 | INFO     | src.beam.experiment:__init__:189 - identifier: debug
2022-06-26 15:27:13.754 | INFO     | src.beam.experiment:__init__:189 - algorithm: CovtypeAlgorithm
2022-06-26 15:27:13.755 | INFO     | src.beam.experiment:__init__:189 - mp_port: None
2022-06-26 15:27:13.756 | INFO     | src.beam.experiment:__init__:189 - root_dir: /home/shared/data/results/covtype
2022-06-26 15:27:13.756 | INFO     | src.beam.experiment:__init__:189 - reload: False
2022-06-26 15:27:13.757 | INFO     | src.beam.experiment:__init__:189 - resume: -1
2022-06-26 15:27:13.757 | INFO     | src.beam.experiment:__init__:189 - override: False
2022-06-26 15:27:13.758 | INFO     | src.beam.experiment:__init_

In [38]:
# alg = covtype_algorithm_generator(experiment)

In [39]:
alg = experiment(covtype_algorithm_generator)

2022-06-26 15:27:16.726 | INFO     | src.beam.experiment:run:553 - Single worker mode
2022-06-26 15:27:16.728 | INFO     | src.beam.experiment:run_worker:45 - Worker: 1/1 is running...


train:   0%|          | 0/339 [00:00<?, ?it/s]

validation:   0%|          | 0/112 [00:00<?, ?it/s]

2022-06-26 15:29:03.183 | INFO     | src.beam.model:step:590 - br was changed to 1
2022-06-26 15:29:03.184 | INFO     | src.beam.model:step:591 - lambda_llr was changed to 0

2022-06-26 15:29:03.185 | INFO     | src.beam.experiment:save_model_results:397 - Finished epoch 1/100:
2022-06-26 15:29:03.189 | INFO     | src.beam.experiment:log_data:443 - train:
2022-06-26 15:29:03.190 | INFO     | src.beam.experiment:log_data:447 - loss 1046.82 	|
2022-06-26 15:29:03.191 | INFO     | src.beam.experiment:log_data:447 - acc 0.705839 	|
2022-06-26 15:29:03.191 | INFO     | src.beam.experiment:log_data:447 - lr 0.001 	|
2022-06-26 15:29:03.192 | INFO     | src.beam.experiment:log_data:443 - validation:
2022-06-26 15:29:03.193 | INFO     | src.beam.experiment:log_data:447 - loss 925.4 	|
2022-06-26 15:29:03.193 | INFO     | src.beam.experiment:log_data:447 - acc 0.764056 	|


train:   0%|          | 0/339 [00:00<?, ?it/s]

...

2022-06-26 18:21:45.808 | INFO     | src.beam.model:step:590 - br was changed to 0.8305630072461575
2022-06-26 18:21:45.809 | INFO     | src.beam.model:step:591 - lambda_llr was changed to 0.08722458820583187

2022-06-26 18:21:45.811 | INFO     | src.beam.experiment:save_model_results:397 - Finished epoch 99/100:
2022-06-26 18:21:45.814 | INFO     | src.beam.experiment:log_data:443 - train:
2022-06-26 18:21:45.815 | INFO     | src.beam.experiment:log_data:447 - loss 691.06 	|
2022-06-26 18:21:45.816 | INFO     | src.beam.experiment:log_data:447 - acc 0.890544 	|
2022-06-26 18:21:45.817 | INFO     | src.beam.experiment:log_data:447 - lr 1e-05 	|
2022-06-26 18:21:45.818 | INFO     | src.beam.experiment:log_data:443 - validation:
2022-06-26 18:21:45.818 | INFO     | src.beam.experiment:log_data:447 - loss 707.328 	|
2022-06-26 18:21:45.819 | INFO     | src.beam.experiment:log_data:447 - acc 0.882446 	|


train:   0%|          | 0/339 [00:00<?, ?it/s]

validation:   0%|          | 0/112 [00:00<?, ?it/s]

2022-06-26 18:23:31.715 | INFO     | src.beam.model:step:590 - br was changed to 0.8290065502177499
2022-06-26 18:23:31.716 | INFO     | src.beam.model:step:591 - lambda_llr was changed to 0.08819784233820116

2022-06-26 18:23:31.718 | INFO     | src.beam.experiment:save_model_results:397 - Finished epoch 100/100:
2022-06-26 18:23:31.722 | INFO     | src.beam.experiment:log_data:443 - train:
2022-06-26 18:23:31.723 | INFO     | src.beam.experiment:log_data:447 - loss 693.034 	|
2022-06-26 18:23:31.724 | INFO     | src.beam.experiment:log_data:447 - acc 0.889118 	|
2022-06-26 18:23:31.725 | INFO     | src.beam.experiment:log_data:447 - lr 1e-05 	|
2022-06-26 18:23:31.727 | INFO     | src.beam.experiment:log_data:443 - validation:
2022-06-26 18:23:31.728 | INFO     | src.beam.experiment:log_data:447 - loss 708.265 	|
2022-06-26 18:23:31.729 | INFO     | src.beam.experiment:log_data:447 - acc 0.883493 	|


*** SIGTERM received at time=1656335415 on cpu 73 ***
PC: @     0x7f4da3f70155  (unknown)  __clone
    @     0x7f4da41bd3c0  1257294768  (unknown)
    @     0x7f4da3b25bb0  (unknown)  (unknown)
    @     0x563542705fa0  (unknown)  (unknown)
[2022-06-27 13:10:15,855 E 10960 50473] logging.cc:325: *** SIGTERM received at time=1656335415 on cpu 73 ***
[2022-06-27 13:10:15,855 E 10960 50473] logging.cc:325: PC: @     0x7f4da3f70155  (unknown)  __clone
[2022-06-27 13:10:15,860 E 10960 50473] logging.cc:325:     @     0x7f4da41bd3c0  1257294768  (unknown)
[2022-06-27 13:10:15,866 E 10960 50473] logging.cc:325:     @     0x7f4da3b25bb0  (unknown)  (unknown)
[2022-06-27 13:10:15,875 E 10960 50473] logging.cc:325:     @     0x563542705fa0  (unknown)  (unknown)


In [13]:
results = alg.evaluate('test')

test:   0%|          | 0/454 [00:00<?, ?it/s]

In [15]:
np.mean(results['scalar']['acc'])

0.8330432722747063

In [14]:
%load_ext tensorboard

In [15]:
%tensorboard --logdir /home/shared/data/results/covtype/covtype/CovtypeAlgorithm/debug/0001_20220622_160022 --port=17067 --bind_all

In [6]:
args = beam_arguments(
        f"--project-name=covtype --root-dir={root_dir} --algorithm=CovtypeAlgorithm --device=1 --no-amp --lr-d=1e-3 --batch-size=256",
        "--n-epochs=100 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark --identifier=half_precision",
        "--weight-decay=1e-5 --beta1=0.9 --beta2=0.999", label_smoothing=.05, weight_factor=.5,
        path_to_data=path_to_data, gamma=1., dropout=.0, activation='gelu', channels=512)

experiment = Experiment(args)

2022-06-23 11:44:34.677 | INFO     | src.beam.experiment:__init__:183 - beam project: covtype
2022-06-23 11:44:34.679 | INFO     | src.beam.experiment:__init__:184 - Experiment Hyperparameters
2022-06-23 11:44:34.683 | INFO     | src.beam.experiment:__init__:189 - project_name: covtype
2022-06-23 11:44:34.685 | INFO     | src.beam.experiment:__init__:189 - identifier: half_precision
2022-06-23 11:44:34.686 | INFO     | src.beam.experiment:__init__:189 - algorithm: CovtypeAlgorithm
2022-06-23 11:44:34.687 | INFO     | src.beam.experiment:__init__:189 - mp_port: None
2022-06-23 11:44:34.688 | INFO     | src.beam.experiment:__init__:189 - root_dir: /home/shared/data/results/covtype
2022-06-23 11:44:34.688 | INFO     | src.beam.experiment:__init__:189 - reload: False
2022-06-23 11:44:34.689 | INFO     | src.beam.experiment:__init__:189 - resume: -1
2022-06-23 11:44:34.690 | INFO     | src.beam.experiment:__init__:189 - override: False
2022-06-23 11:44:34.690 | INFO     | src.beam.experimen

In [7]:
alg = experiment(covtype_algorithm_generator)

2022-06-23 11:44:37.677 | INFO     | src.beam.experiment:run:553 - Single worker mode
2022-06-23 11:44:37.680 | INFO     | src.beam.experiment:run_worker:45 - Worker: 1/1 is running...


train:   0%|          | 0/1360 [00:00<?, ?it/s]

validation:   0%|          | 0/452 [00:00<?, ?it/s]


2022-06-23 11:50:32.263 | INFO     | src.beam.experiment:save_model_results:397 - Finished epoch 1/100:
2022-06-23 11:50:32.272 | INFO     | src.beam.experiment:log_data:443 - train:
2022-06-23 11:50:32.274 | INFO     | src.beam.experiment:log_data:447 - loss 228.893 	|
2022-06-23 11:50:32.274 | INFO     | src.beam.experiment:log_data:447 - acc 0.695109 	|
2022-06-23 11:50:32.275 | INFO     | src.beam.experiment:log_data:447 - lr 0.001 	|
2022-06-23 11:50:32.277 | INFO     | src.beam.experiment:log_data:443 - validation:
2022-06-23 11:50:32.283 | INFO     | src.beam.experiment:log_data:447 - loss 195.103 	|
2022-06-23 11:50:32.288 | INFO     | src.beam.experiment:log_data:447 - acc 0.758547 	|


train:   0%|          | 0/1360 [00:00<?, ?it/s]

...

train:   0%|          | 0/1360 [00:00<?, ?it/s]

validation:   0%|          | 0/452 [00:00<?, ?it/s]


2022-06-23 12:14:43.038 | INFO     | src.beam.experiment:save_model_results:397 - Finished epoch 5/100:
2022-06-23 12:14:43.057 | INFO     | src.beam.experiment:log_data:443 - train:
2022-06-23 12:14:43.058 | INFO     | src.beam.experiment:log_data:447 - loss nan 	|
2022-06-23 12:14:43.060 | INFO     | src.beam.experiment:log_data:447 - acc 0.903142 	|
2022-06-23 12:14:43.060 | INFO     | src.beam.experiment:log_data:447 - lr 0.001 	|
2022-06-23 12:14:43.062 | INFO     | src.beam.experiment:log_data:443 - validation:
2022-06-23 12:14:43.063 | INFO     | src.beam.experiment:log_data:447 - loss nan 	|
2022-06-23 12:14:43.064 | INFO     | src.beam.experiment:log_data:447 - acc 0 	|


train:   0%|          | 0/1360 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Hyperparameter optimization with ray and optuna

In [3]:
args = beam_arguments(get_covtype_parser(), 
        f"--project-name=covtype --root-dir={root_dir} --algorithm=CovtypeAlgorithm --device=0 --no-half --lr-d=1e-3 --batch-size=256",
        "--n-epochs=40 --clip=0 --parallel=1 --accumulate=1 --cudnn-benchmark",
        "--weight-decay=1e-5 --beta1=0.9 --beta2=0.999", label_smoothing=.05, weight_factor=.0,
        path_to_data=path_to_data, dropout=.0, activation='gelu', channels=128, n_rules=64, n_layers=2,)

study = Study(covtype_algorithm_generator, args)

2022-06-23 16:26:10.599 | INFO     | src.beam.experiment:__init__:82 - Hyperparameter Optimization
2022-06-23 16:26:10.600 | INFO     | src.beam.experiment:__init__:83 - beam project: covtype
2022-06-23 16:26:10.601 | INFO     | src.beam.experiment:__init__:84 - Experiment Hyperparameters
2022-06-23 16:26:10.601 | INFO     | src.beam.experiment:__init__:87 - project_name: covtype
2022-06-23 16:26:10.602 | INFO     | src.beam.experiment:__init__:87 - identifier: debug_hp_optimization_20220623_162610
2022-06-23 16:26:10.602 | INFO     | src.beam.experiment:__init__:87 - algorithm: CovtypeAlgorithm
2022-06-23 16:26:10.603 | INFO     | src.beam.experiment:__init__:87 - mp_port: None
2022-06-23 16:26:10.603 | INFO     | src.beam.experiment:__init__:87 - root_dir: /home/shared/data/results/covtype
2022-06-23 16:26:10.603 | INFO     | src.beam.experiment:__init__:87 - reload: False
2022-06-23 16:26:10.604 | INFO     | src.beam.experiment:__init__:87 - resume: -1
2022-06-23 16:26:10.604 | INFO

In [4]:
analysis = study.tune(config={"lr_dense": tune.loguniform(1e-4, 1e-2),
                              "lr_sparse": tune.loguniform(1e-3, 1e-1),
                              "dropout": tune.uniform(.0, .25),
                              "scheduler_factor": tune.uniform(.1, .5),
                              "scheduler_patience": tune.choice([12, 16, 20]),
                              "weight_decay": tune.loguniform(1e-6, 1e-3),
                              "beta1": tune.choice([.9, .95]),
                              "beta2": tune.choice([.99, .999]),
                              "n_tables": tune.choice([1, 5, 15]),
                              "n_quantiles": tune.choice([4, 10, 20]),
                              "weight_factor": tune.choice([.0, .25, .5]),
                              "label_smoothing": tune.choice([.0, .1, .2]),
                              "channels": tune.choice([64, 128, 256]),
                              "n_rules": tune.choice([64, 128, 256]),
                              "n_layers": tune.choice([1, 2]),
                              "activation": tune.choice(['celu', 'gelu']),
                              "k_p": tune.loguniform(.01, .1),
                              "k_i": tune.loguniform(.001, .01),
                              "k_d": tune.loguniform(.001, .01),
                              "initial_mask": tune.uniform(.9, 1.),},
                       metric="mean_accuracy",
                       max_concurrent_trials=2,
                       resources_per_trial={"gpu": 1},
                       mode="max",
                       search_alg=OptunaSearch(),
                      progress_reporter=JupyterNotebookReporter(overwrite=True),
                       num_samples=1000)

Trial name,status,loc,activation,beta1,beta2,channels,dropout,initial_mask,k_d,k_i,k_p,label_smoothing,lr_dense,lr_sparse,n_layers,n_quantiles,n_rules,n_tables,scheduler_factor,scheduler_patience,weight_decay,weight_factor,acc,iter,total time (s)
runner_tune_63c3d776,RUNNING,172.17.0.7:66237,gelu,0.9,0.999,256,0.058916,0.999251,0.00353359,0.00155413,0.0555205,0.2,0.00217149,0.017969,2,20,128,15,0.135275,12,0.000351825,0,0.948467,36,5490.23
runner_tune_edaad5d2,RUNNING,172.17.0.7:67440,gelu,0.9,0.999,256,0.0494754,0.998241,0.00445461,0.0012865,0.0747774,0.2,0.000941906,0.0200523,2,20,64,15,0.122971,12,0.00038737,0,0.950014,29,2290.66
runner_tune_008a91a0,TERMINATED,172.17.0.7:50284,gelu,0.9,0.99,256,0.0807869,0.996324,0.00680326,0.00142563,0.0510722,0.2,0.000819026,0.0152461,2,20,128,15,0.112404,12,0.000984329,0,0.953592,40,6066.92
runner_tune_028e4942,TERMINATED,172.17.0.7:20640,gelu,0.9,0.99,64,0.138117,0.98606,0.00797104,0.00847916,0.0140101,0.2,0.000334277,0.050323,1,20,256,1,0.153175,16,1.73845e-06,0,0.93171,40,5513.77
runner_tune_080099b0,TERMINATED,172.17.0.7:80084,celu,0.9,0.999,64,0.158326,0.996888,0.00856642,0.00988607,0.0889488,0.1,0.00986728,0.0959956,2,20,256,1,0.1056,16,1.25602e-06,0,0.912118,40,10459.7
runner_tune_0e789ebc,TERMINATED,172.17.0.7:60721,celu,0.9,0.999,256,0.0295621,0.999882,0.00456891,0.00102814,0.0707723,0.2,0.000994112,0.0304406,2,4,64,15,0.100448,12,0.000715209,0,0.947706,40,2903.16
runner_tune_10fb7f6c,TERMINATED,172.17.0.7:72407,gelu,0.9,0.99,256,0.220779,0.968569,0.00308747,0.002626,0.0240213,0.1,0.000142814,0.00126458,1,10,256,5,0.267074,20,1.47414e-05,0.25,0.896104,40,8462.36
runner_tune_121f98fc,TERMINATED,172.17.0.7:40384,gelu,0.95,0.99,256,0.1964,0.963854,0.00830381,0.0019387,0.0374598,0.1,0.00100392,0.00268295,2,10,128,5,0.226368,20,3.64052e-05,0,0.933931,40,5975.81
runner_tune_1fc0b7b0,TERMINATED,172.17.0.7:38270,celu,0.95,0.999,256,0.185739,0.982717,0.00635819,0.00250672,0.0167592,0,0.00241463,0.0140612,2,10,128,5,0.165462,20,1.75448e-05,0,0.928149,40,5984.78
runner_tune_20003a20,TERMINATED,172.17.0.7:14600,celu,0.9,0.99,64,0.189547,0.979488,0.0099452,0.00441909,0.0125956,0.2,0.000193391,0.0418173,1,20,256,15,0.190423,16,9.5758e-06,0,0.921564,40,5645.03


2022-06-26 06:22:32,947	WARNING tune.py:650 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


KeyboardInterrupt: 

(runner_tune pid=67440) 2022-06-26 06:23:07.687 | INFO     | src.beam.model:step:584 - br was changed to 0.9798078372723117
(runner_tune pid=67440) 2022-06-26 06:23:07.687 | INFO     | src.beam.model:step:585 - lambda_llr was changed to 0.017812550184276018
(runner_tune pid=66237) 2022-06-26 06:23:49.308 | INFO     | src.beam.model:step:584 - br was changed to 0.9730077730625055
(runner_tune pid=66237) 2022-06-26 06:23:49.308 | INFO     | src.beam.model:step:585 - lambda_llr was changed to 0.024606548212574783
